In [ ]:
## Test Quote aggregation
import sys
from config import config
sys.path.insert(0,'./GenderGapTracker/nlp/english')
from quote_extractor import QuoteExtractor
import spacy
import pandas as pd
nlp = spacy.load('en_core_web_lg')

qt = QuoteExtractor(config)

In [ ]:
text = '''

Senator Matt Canavan, an ally of new Nationals leader Barnaby Joyce, has accused the National Farmers' Federation of hypocrisy for advocating no net emissions of greenhouse gases by 2050 while objecting to land-clearing rules that have helped reduce Australia's carbon dioxide emissions.

The criticism reflects a split within agriculture-based organisations about the benefits or harm of Australia committing to what is known as the CN2050 target at a November United Nations climate conference in Glasgow.

"I find their position absolutely incoherent, the way they have us reducing emissions by 50 per cent through government locking up huge areas of land through tree-clearing laws, which the NFF are opposed to," Senator Canavan, the Nationals' deputy Senate leader, said in an interview.

"The reason we are meeting Kyoto [emissions targets] with a spillover - that is all from 6 million hectares of pastoral land reforested from land-use regulations.

"I suppose they are not the only political organisation to have hypocritical policy positions."

Some National Party politicians, especially in Victoria, have expressed concerns that under Mr Joyce, who was re-elected leader on June 21, the party will become reluctant to support or agree to more stringent climate policies.
'''
doc = nlp(text)
quotes_o = qt.extract_quotes(doc=doc)

In [ ]:
def quote_start(quote):
    '''
    Combine all spans of speaker, quote and verb and output the start of the 
    '''
    names = ['speaker_index', 'quote_index', 'verb_index']
    span = ()
    for n in names:
        if sum(eval(quote[n])):
            span += eval(quote[n])
    return min(span)
    
quotes_s = sorted(quotes_o, key=lambda q: quote_start(q))

In [ ]:
def valid_speaker(q):
    return not (q["speaker_index"] == '(0,0)' or q["speaker_pos"] == ['PRON'])

def aggregate(group):
    if len(group) <= 1:
        return group
    else:
        result = group[0]
        result['quote'] = ' \n'.join([q['quote'] for q in group]) 
        result['quote_index'] = ', '.join([q['quote_index'] for q in group])
        result['verb'] = ', '.join([q['verb'] for q in group])
        result['verb_index'] = ', '.join([q['verb_index'] for q in group])
        result['quote_token_count'] = sum([q['quote_token_count'] for q in group])
        result['quote_type'] = ', '.join([q['quote_type'] for q in group])
        return [result]

def aggregate_quotes(quotes):
    results = []
    group = []
    for q in quotes:
        if valid_speaker(q):
            results.extend(aggregate(group)) # If the current quote has a valid speaker, aggregate the previous quote group.
            group = [q]
        else:
            group.append(q)
    results.extend(aggregate(group)) # Aggregate the very last group of quotes
    return results
quotes_a = aggregate_quotes(quotes_s)

In [ ]:
with pd.ExcelWriter("extracted_quotes.xlsx") as writer:
    pd.DataFrame.from_dict(quotes_o).to_excel(writer, sheet_name="Orig_quotes")  
    pd.DataFrame.from_dict(quotes_s).to_excel(writer, sheet_name="Sorted_quotes")
    pd.DataFrame.from_dict(quotes_a).to_excel(writer, sheet_name="Aggregated_quotes") 